In [ ]:
# @title 📊 Interfaz para Concatenar Columnas
# --- Importaciones necesarias ---
import pandas as pd
import ipywidgets as widgets
from google.colab import files
from IPython.display import display, clear_output
import io

# --- Instalación silenciosa de la dependencia para .xlsx ---
!pip install -q openpyxl

# --- Clase principal de la aplicación ---
class VisualConcatenator:
    def __init__(self):
        """Inicializa la interfaz y sus componentes."""
        self.df = None
        self.output_df = None

        # --- Definición de los Widgets (Componentes de la UI) ---
        self.title = widgets.HTML("<h2>🔗 Concatenador Visual de Columnas</h2><hr>")
        self.instructions = widgets.HTML("<h4>Paso 1: Sube tu archivo Excel para empezar.</h4>")

        self.uploader = widgets.FileUpload(
            accept='.xlsx',
            description='Subir Archivo',
            button_style='primary'
        )

        # Contenedor para la segunda parte de la UI (que aparece después de subir)
        self.step2_container = widgets.VBox([])

        # Área de salida para mensajes y resultados
        self.output_area = widgets.Output()

        # --- Observador de eventos ---
        # Llama a la función _handle_upload cuando se sube un archivo
        self.uploader.observe(self._handle_upload, names='value')

    def _handle_upload(self, change):
        """Se activa al subir un archivo. Lee el archivo y crea la UI de selección."""
        # Limpia cualquier contenido previo en el área de salida
        with self.output_area:
            clear_output()

        uploaded_file_info = self.uploader.value
        if not uploaded_file_info:
            return # Si se cancela la subida, no hacer nada

        # Obtener el nombre y contenido del archivo subido
        filename = list(uploaded_file_info.keys())[0]
        content = uploaded_file_info[filename]['content']

        try:
            with self.output_area:
                print(f"⏳ Leyendo '{filename}'...")
                # Leer el archivo Excel desde la memoria
                self.df = pd.read_excel(io.BytesIO(content))
                print(f"✅ ¡Archivo cargado! Se encontraron {self.df.shape[0]} filas.")
                print(f"📊 Columnas disponibles: {', '.join(self.df.columns)}")

            # Si el archivo se leyó bien, crear la siguiente parte de la interfaz
            self._create_column_selector_ui()

        except Exception as e:
            with self.output_area:
                print(f"❌ Error al leer el archivo. Asegúrate de que es un .xlsx válido.")
                print(f"Detalle: {e}")

    def _create_column_selector_ui(self):
        """Crea los widgets para seleccionar columnas y procesar."""

        # Nuevas instrucciones para el usuario
        step2_instructions = widgets.HTML("""
        <h4>Paso 2: Selecciona las columnas a unir.</h4>
        <p><i>Usa <b>Ctrl+Click</b> (o <b>Cmd+Click</b> en Mac) para elegir varias.</i></p>
        """)

        # Widget para seleccionar múltiples columnas
        self.column_selector = widgets.SelectMultiple(
            options=self.df.columns.tolist(),
            description='Columnas:',
            layout=widgets.Layout(height='180px', width='95%'),
            style={'description_width': 'initial'}
        )

        # Botón para iniciar el procesamiento
        self.process_button = widgets.Button(
            description='🔗 Procesar y Descargar Resultado',
            button_style='success',
            icon='cogs',
            layout=widgets.Layout(width='300px', height='40px', margin='10px 0 0 0')
        )

        # Conectar el botón a la función de procesamiento
        self.process_button.on_click(self._process_and_download)

        # Actualizar el contenedor del Paso 2 con los nuevos widgets
        self.step2_container.children = [
            step2_instructions,
            self.column_selector,
            self.process_button
        ]

    def _process_and_download(self, b):
        """Realiza la concatenación, acortado y descarga del archivo."""
        with self.output_area:
            clear_output()

            selected_cols = self.column_selector.value
            if len(selected_cols) < 2:
                print("❌ Error: Debes seleccionar al menos dos columnas para poder unirlas.")
                return

            print("🚀 Procesando los datos...")
            print(f"   - Uniendo columnas: {', '.join(selected_cols)}")

            try:
                # --- Lógica principal ---
                # 1. Función para acortar el texto a 80 palabras
                def acortar_texto(texto, limite=80):
                    palabras = str(texto).split()
                    if len(palabras) > limite:
                        return ' '.join(palabras[:limite]) + '...'
                    return ' '.join(palabras) # .join() limpia espacios múltiples

                # 2. Convertir columnas a texto, rellenar vacíos y unir con un espacio
                texto_unido = self.df[list(selected_cols)].fillna('').astype(str).apply(
                    lambda fila: ' '.join(fila), axis=1
                )

                # 3. Acortar el texto ya unido
                texto_acortado = texto_unido.apply(acortar_texto)

                # 4. Crear el DataFrame final con una única columna llamada "resumen"
                self.output_df = pd.DataFrame({'resumen': texto_acortado})

                print("\n✅ ¡Procesamiento completado con éxito!")

                # --- Lógica de descarga ---
                output_filename = 'resumen_concatenado.xlsx'
                print(f"\n📦 Generando el archivo '{output_filename}' para descarga...")

                # Guardar el DataFrame en un archivo Excel dentro del entorno de Colab
                self.output_df.to_excel(output_filename, index=False, engine='openpyxl')

                # Usar la función de Colab para iniciar la descarga en el navegador
                files.download(output_filename)

            except Exception as e:
                print(f"\n❌ Ocurrió un error inesperado durante el procesamiento.")
                print(f"Detalle: {e}")

    def display_app(self):
        """Muestra la aplicación completa en la celda."""
        display(
            self.title,
            self.instructions,
            self.uploader,
            self.step2_container,
            self.output_area
        )

# --- Punto de entrada: Crear y mostrar la aplicación ---
app = VisualConcatenator()
app.display_app()

HTML(value='<h2>🔗 Concatenador Visual de Columnas</h2><hr>')

HTML(value='<h4>Paso 1: Sube tu archivo Excel para empezar.</h4>')

FileUpload(value={}, accept='.xlsx', button_style='primary', description='Subir Archivo')

VBox()

Output()

# Analizador v2.0 HÍBRIDO
## ML (E5) para clustering semántico → Agrupa similares
##📏 Reglas lingüísticas para tono → Explicable y controlable
##🏷️ Keywords para tema → Adaptable al dominio

In [ ]:
# @title 🔧 Instalaciones
!pip install -q torch transformers sentence-transformers pandas openpyxl tqdm scikit-learn 2>/dev/null


In [11]:
# @title Analizador v2.0

# @markdown • Análisis de TONO
# @markdown • NO usa modelo de ML - Usa reglas lingüísticas

import os, re, time, torch
import numpy as np
import pandas as pd
import warnings
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from google.colab import files

warnings.filterwarnings("ignore")

# ==================== CONFIGURACIÓN ====================
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 Dispositivo: {device.upper()}")

# Modelo multilingüe con licencia Apache 2.0
embedder = SentenceTransformer("intfloat/multilingual-e5-large", device=device)

# ==================== REGLAS MEJORADAS ====================
POSITIVE_VERBS_CLIENT = [
    "confirma", "anuncia", "inaugura", "lanza", "lidera", "impulsa", "acompaña","continua",
    "gestiona", "acompaña", "otorga", "destina", "facilita", "apoya", "confirma","recibe",
    "reconoce", "homenajea", "firma", "inicia", "invierte", "entrega", "adelanta","demuestra",
    "presenta", "logra", "avanza", "atiende", "visita", "verifica","continúa","intensifica",
    "construye", "desarrolla", "fortalece", "promueve", "garantiza","expresa",
]

# Contextos que indican gestión POSITIVA del cliente
POSITIVE_CONTEXTS = [
    "todo está listo", "obras de", "inversión en","recibe distinción",
    "mejoras en", "avance en", "beneficio", "articulación","expresa solidaridad",
    "liderazgo del", "bajo el liderazgo", "gracias a la alcaldía",
    "gracias al alcalde", "proyecto es posible gracias",
    "gestión de", "gestión del alcalde"
]

# Logros de programas/proyectos que implican gestión positiva
ACHIEVEMENT_INDICATORS = [
    "bachilleres inician", "bachilleres se han inscrito", "estudiantes inician",
    "familias beneficiadas", "viviendas entregadas", "obras culminadas",
    "proyectos en marcha", "inversión de", "beneficiarios",
    "acceso a", "programas profesionales", "vida universitaria"
]

# Palabras que indican acciones REACTIVAS (neutras, no gestión)
NEUTRAL_REACTIONS = [
    "lamenta", "rechaza", "condena", "expresa solidaridad",
    "pide", "solicita", "exige", "manifesta preocupación"
]

# Contextos claramente negativos
NEGATIVE_TRIGGERS = [
    "escándalo", "corrupción", "investigan al", "denuncian al",
    "irregularidad", "desfalco", "acusan", "protesta contra",
    "demanda contra", "sancionan", "fraude", "malversación"
]

TOPICS_KEYWORDS = {
    "Infraestructura": ["escenario", "multideportivo", "obra", "infraestructura", "vía", "parque",
                        "pavimentación", "construcción", "puente", "polideportivo", "cancha"],
    "Educación": ["educación", "bachilleres", "universitaria", "colegio", "becas", "estudiantes",
                  "escuela", "universidad", "educación superior", "acceso a la universidad"],
    "Seguridad y Justicia": ["policía", "seguridad", "auxiliares", "dotación", "homicidio",
                            "delincuencia", "crimen", "atentado", "fallecimiento"],
    "Relaciones Internacionales": ["embajador", "cumbre", "celac", "diplomático", "internacional", "bilateral"],
    "Corrupción y Escándalos": ["escándalo", "corrupción", "investigan al", "denuncia contra", "fraude"],
    "Salud": ["salud", "hospital", "médico", "atención médica", "vacunación", "pandemia"],
    "Medio Ambiente": ["lluvias", "inundaciones", "emergencia", "clima", "ambiental", "desastre natural",
                      "huracán", "afectados", "coletazo"],
    "Economía": ["economía", "empleo", "empresa", "comercio", "inversión económica", "presupuesto"],
}

# ==================== FUNCIONES CORREGIDAS ====================

def normalize_text(text):
    """Normaliza texto removiendo artículos y preposiciones para mejor clustering"""
    # Convertir a minúsculas
    t = text.lower()

    # Remover artículos y preposiciones comunes
    stopwords = ['el', 'la', 'los', 'las', 'un', 'una', 'unos', 'unas',
                 'de', 'del', 'en', 'para', 'por', 'con', 'sin', 'sobre',
                 'que', 'todo', 'está', 'son', 'es']

    # Mantener solo caracteres alfanuméricos y espacios
    t = re.sub(r'[^\wáéíóúñ\s]', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()

    # Remover stopwords
    words = t.split()
    words = [w for w in words if w not in stopwords and len(w) > 2]

    return " ".join(words)

def get_signature(text, words=20):
    """Genera firma textual normalizada para clustering"""
    normalized = normalize_text(text)
    # Tomar más palabras para capturar mejor la similitud
    return " ".join(normalized.split()[:words])

def universal_cluster(texts):
    """Clustering REAL - noticias similares tendrán mismo tema/tono"""
    print("📊 Agrupando noticias similares...")
    groups = defaultdict(list)

    for i, t in enumerate(texts):
        signature = get_signature(t)
        groups[signature].append(i)

    representatives = []
    cluster_map = [-1] * len(texts)
    cid = 0

    for idxs in groups.values():
        # El representante es el texto más largo del grupo (más completo)
        longest_idx = max(idxs, key=lambda i: len(texts[i]))
        representatives.append(texts[longest_idx])

        for i in idxs:
            cluster_map[i] = cid
        cid += 1

    print(f"✅ {len(texts)} noticias → {len(representatives)} grupos únicos")
    return representatives, np.array(cluster_map)

def analyze_universal(rep_text, client_name):
    """Análisis MEJORADO con lógica de contexto"""
    lower = rep_text.lower()
    client_lower = client_name.lower()

    # === ANÁLISIS DE TONO CON PRIORIDADES ===

    # PRIORIDAD 1: Negativos directos contra el cliente
    negative_about_client = any(
        f"{neg} {client_lower}" in lower or f"{client_lower} {neg}" in lower
        for neg in ["investigan al", "denuncian al", "acusan", "escándalo", "corrupción"]
    )

    if negative_about_client:
        tono = "Negativo"
        confianza = 0.95

    # PRIORIDAD 2: Reacciones neutras (lamenta, rechaza eventos externos)
    elif any(reaction in lower for reaction in NEUTRAL_REACTIONS):
        # Si solo reacciona a eventos externos = Neutro
        # (no es gestión propia, solo declaraciones)
        tono = "Neutro"
        confianza = 0.88

    # PRIORIDAD 3: Gestión positiva del cliente
    else:
        # Buscar si el cliente ES EL GESTOR (sujeto activo)
        is_active_subject = False

        # Contexto 1: "liderazgo del alcalde X", "gracias a X"
        leadership_patterns = [
            f"liderazgo del {client_lower}",
            f"liderazgo de {client_lower}",
            f"gracias a {client_lower}",
            f"gracias al {client_lower}",
            f"gestión del {client_lower}",
            f"gestión de {client_lower}"
        ]

        if any(pattern in lower for pattern in leadership_patterns):
            is_active_subject = True

        # Contexto 2: "alcalde X + verbo de gestión activa"
        if not is_active_subject:
            active_verbs = ["confirma", "anuncia", "inaugura", "entrega", "invierte",
                          "construye", "lidera", "impulsa", "gestiona", "inicia"]

            for verb in active_verbs:
                # Patrón: "alcalde X verbo"
                if f"alcalde {client_lower} {verb}" in lower or \
                   f"alcaldía {verb}" in lower:
                    is_active_subject = True
                    break

        # Contexto 3: Logros/resultados de programas (implican gestión)
        # "500 bachilleres inician...", "familias beneficiadas..."
        has_achievement = any(indicator in lower for indicator in ACHIEVEMENT_INDICATORS)

        # Contexto 4: Presencia de indicadores positivos generales
        has_positive_context = any(ctx in lower for ctx in POSITIVE_CONTEXTS)

        # Si hay logros concretos en áreas de gestión pública = Positivo
        if is_active_subject or has_positive_context or has_achievement:
            tono = "Positivo"
            confianza = 0.92 if is_active_subject else 0.85
        else:
            tono = "Neutro"
            confianza = 0.70

    # === ANÁLISIS DE TEMA ===
    tema = "Gestión y Acciones"  # Por defecto
    max_matches = 0

    for topic, keywords in TOPICS_KEYWORDS.items():
        matches = sum(1 for kw in keywords if kw in lower)
        if matches > max_matches:
            max_matches = matches
            tema = topic

    return tono, round(confianza, 3), tema

# ==================== MAIN EXECUTION ====================
print("\n" + "="*80)
print("📰 ANALIZADOR DE NOTICIAS v30 - CLUSTERING REAL")
print("="*80 + "\n")

# Solicitar nombre del cliente
client_name = input("👤 Nombre del cliente (ej: Carlos Pinedo): ").strip()
if not client_name:
    client_name = "Carlos Pinedo"
    print(f"   Usando cliente por defecto: {client_name}")

# Cargar archivo
print("\n📂 Sube tu archivo Excel...")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Leer datos
print(f"📖 Leyendo {file_name}...")
df = pd.read_excel(file_name)

# Detectar columna de texto
text_columns = ['resumen', 'texto', 'contenido', 'titulo', 'title', 'noticia', 'descripcion']
text_col = next((c for c in text_columns if c in df.columns), None)

if not text_col:
    print(f"❌ Error: No se encontró columna de texto. Columnas disponibles: {list(df.columns)}")
    raise ValueError("No se encontró columna de texto válida")

print(f"✅ Columna detectada: '{text_col}'")

# Limpiar datos
df = df.dropna(subset=[text_col]).reset_index(drop=True)
texts = df[text_col].astype(str).tolist()
print(f"📊 {len(texts)} noticias cargadas\n")

# ==================== PROCESAMIENTO ====================
t0 = time.time()

# Clustering REAL
rep_texts, cluster_map = universal_cluster(texts)

# Análisis - CADA GRUPO SE ANALIZA UNA SOLA VEZ
print("\n🔍 Analizando tono y tema de grupos únicos...")
tonos = []
confianzas = []
temas = []

for rep in tqdm(rep_texts, desc="Procesando"):
    tono, conf, tema = analyze_universal(rep, client_name)
    tonos.append(tono)
    confianzas.append(conf)
    temas.append(tema)

# Mapear resultados - TODAS las noticias del mismo grupo tendrán EXACTAMENTE el mismo tema/tono
print("\n🔗 Aplicando tema/tono a grupos similares...")
df["grupo_id"] = cluster_map
df["tono_marca"] = [tonos[cluster_map[i]] for i in range(len(df))]
df["confianza"] = [confianzas[cluster_map[i]] for i in range(len(df))]
df["tema"] = [temas[cluster_map[i]] for i in range(len(df))]

# Verificación: Mostrar algunos grupos para debugging
print("\n🔍 Verificando grupos (muestra):")
for gid in sorted(df["grupo_id"].unique())[:3]:
    grupo = df[df["grupo_id"] == gid]
    if len(grupo) > 1:
        print(f"\n   Grupo {gid} ({len(grupo)} noticias) - Tono: {tonos[gid]} - Tema: {temas[gid]}")
        for idx, row in grupo.head(2).iterrows():
            texto_corto = row[text_col][:80] + "..." if len(row[text_col]) > 80 else row[text_col]
            print(f"      • {texto_corto}")

# ==================== EXPORTACIÓN (SOLO 1 HOJA) ====================
out = f"Final_{client_name.replace(' ', '_')}_{time.strftime('%Y%m%d_%H%M')}.xlsx"
print(f"\n💾 Guardando resultados en {out}...")

# SOLO UNA HOJA con todos los resultados
df.to_excel(out, sheet_name="Resultados", index=False, engine="openpyxl")

files.download(out)

# ==================== RESULTADOS ====================
tiempo_total = time.time() - t0
print(f"\n{'='*80}")
print(f"✅ ¡ANÁLISIS COMPLETADO!")
print(f"⏱️  Tiempo: {tiempo_total:.1f}s")
print(f"📄 Archivo: {out}")
print(f"{'='*80}\n")

# Estadísticas
total = len(df)
pos = (df["tono_marca"] == "Positivo").sum()
neg = (df["tono_marca"] == "Negativo").sum()
neu = (df["tono_marca"] == "Neutro").sum()

print("📊 RESUMEN:\n")
print(f"   📈 Total noticias: {total}")
print(f"   🔢 Grupos únicos: {len(rep_texts)}")
print(f"   🔗 Promedio noticias/grupo: {total/len(rep_texts):.1f}")
print(f"   ✅ Positivas: {pos} ({pos/total*100:.1f}%)")
print(f"   ❌ Negativas: {neg} ({neg/total*100:.1f}%)")
print(f"   ⚪ Neutras: {neu} ({neu/total*100:.1f}%)")

# Mostrar algunos ejemplos de agrupación
print("\n📋 EJEMPLOS DE AGRUPACIÓN:")
grupos_multi = df[df.duplicated(subset=["grupo_id"], keep=False)].sort_values("grupo_id")
if len(grupos_multi) > 0:
    for gid in grupos_multi["grupo_id"].unique()[:2]:
        grupo = df[df["grupo_id"] == gid]
        print(f"\n   🔗 Grupo {gid} → {len(grupo)} noticias similares")
        print(f"      Tono asignado: {grupo.iloc[0]['tono_marca']}")
        print(f"      Tema asignado: {grupo.iloc[0]['tema']}")
        for idx, row in grupo.head(2).iterrows():
            texto_corto = row[text_col][:70] + "..." if len(row[text_col]) > 70 else row[text_col]
            print(f"      • {texto_corto}")
else:
    print("   ℹ️  No se encontraron noticias duplicadas en esta muestra")

print("\n📈 DISTRIBUCIÓN DE TONO:")
print(df["tono_marca"].value_counts().to_string())

print("\n🎯 DISTRIBUCIÓN DE TEMAS:")
print(df["tema"].value_counts().to_string())

print("\n✨ Análisis finalizado exitosamente ✨")

🚀 Dispositivo: CUDA

📰 ANALIZADOR DE NOTICIAS v30 - CLUSTERING REAL

👤 Nombre del cliente (ej: Carlos Pinedo): Carlos Pinedo

📂 Sube tu archivo Excel...


Saving grok1.xlsx to grok1 (10).xlsx
📖 Leyendo grok1 (10).xlsx...
✅ Columna detectada: 'resumen'
📊 1432 noticias cargadas

📊 Agrupando noticias similares...
✅ 1432 noticias → 1210 grupos únicos

🔍 Analizando tono y tema de grupos únicos...


Procesando:   0%|          | 0/1210 [00:00<?, ?it/s]


🔗 Aplicando tema/tono a grupos similares...

🔍 Verificando grupos (muestra):

💾 Guardando resultados en Final_Carlos_Pinedo_20251118_0228.xlsx...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ ¡ANÁLISIS COMPLETADO!
⏱️  Tiempo: 0.5s
📄 Archivo: Final_Carlos_Pinedo_20251118_0228.xlsx

📊 RESUMEN:

   📈 Total noticias: 1432
   🔢 Grupos únicos: 1210
   🔗 Promedio noticias/grupo: 1.2
   ✅ Positivas: 424 (29.6%)
   ❌ Negativas: 0 (0.0%)
   ⚪ Neutras: 1008 (70.4%)

📋 EJEMPLOS DE AGRUPACIÓN:

   🔗 Grupo 3 → 2 noticias similares
      Tono asignado: Neutro
      Tema asignado: Medio Ambiente
      • Figura del Junior de Barranquilla se puso la diez para ayudar a damnif...
      • Figura del Junior de Barranquilla se puso la diez para ayudar a damnif...

   🔗 Grupo 7 → 2 noticias similares
      Tono asignado: Neutro
      Tema asignado: Salud
      • Hasta el 30 de agosto se tendrán jornadas de esterilización gratuitas ...
      • Hasta el 30 de agosto se tendrán jornadas de esterilización gratuitas ...

📈 DISTRIBUCIÓN DE TONO:
tono_marca
Neutro      1008
Positivo     424

🎯 DISTRIBUCIÓN DE TEMAS:
tema
Medio Ambiente                510
Gestión y Acciones            349
Infraestructu

#Analizador v2.1 HÍBRIDO
##Modelo de sentimiento: RoBERTa-spanish

##Modelo: finiteautomata/beto-sentiment-analysis
##Base: BETO (BERT español)

In [ ]:
# @title 🔧 Instalaciones
!pip install -q torch transformers sentence-transformers pandas openpyxl tqdm scikit-learn 2>/dev/null

In [12]:
# @title Analizador v2.1 HÍBRIDO – RoBERTa (tono) + E5 (clustering)
# @markdown • RoBERTa-spanish para análisis de sentimiento preciso
# @markdown • E5 para clustering de noticias similares
# @markdown • Máxima precisión en tono de marca

!pip install -q torch transformers sentence-transformers pandas openpyxl tqdm scikit-learn 2>/dev/null

import os, re, time, torch
import numpy as np
import pandas as pd
import warnings
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from collections import defaultdict
from google.colab import files

warnings.filterwarnings("ignore")

# ==================== CONFIGURACIÓN ====================
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 Dispositivo: {device.upper()}")

# MODELO 1: E5 para clustering (mantiene agrupación rápida)
print("📦 Cargando multilingual-e5-large para clustering...")
embedder = SentenceTransformer("intfloat/multilingual-e5-large", device=device)

# MODELO 2: RoBERTa-spanish para análisis de sentimiento
print("📦 Cargando RoBERTa-spanish para análisis de tono...")
sentiment_model_name = "finiteautomata/beto-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_name)
sentiment_model = sentiment_model.to(device)
sentiment_model.eval()

print("✅ Modelos cargados\n")

# ==================== REGLAS COMPLEMENTARIAS ====================
# Reglas que REFUERZAN el análisis de RoBERTa

CLIENT_POSITIVE_PATTERNS = [
    "liderazgo del", "bajo el liderazgo", "gracias a la alcaldía",
    "gracias al alcalde", "gestión del alcalde", "alcaldía entrega",
    "alcalde confirma", "alcalde anuncia", "alcalde inaugura"
]

CLIENT_NEGATIVE_PATTERNS = [
    "investigan al alcalde", "denuncian al", "escándalo del alcalde",
    "corrupción del", "acusan al alcalde", "irregularidades del"
]

NEUTRAL_REACTIONS = [
    "lamenta", "rechaza atentado", "condena", "expresa solidaridad",
    "pide", "solicita", "manifiesta preocupación"
]

ACHIEVEMENT_INDICATORS = [
    "bachilleres inician", "estudiantes se inscriben", "familias beneficiadas",
    "viviendas entregadas", "obras culminadas", "inversión de",
    "programas profesionales", "vida universitaria", "acceso a"
]

TOPICS_KEYWORDS = {
    "Infraestructura": ["escenario", "multideportivo", "obra", "infraestructura", "vía", "parque",
                        "pavimentación", "construcción", "puente", "polideportivo", "cancha"],
    "Educación": ["educación", "bachilleres", "universitaria", "colegio", "becas", "estudiantes",
                  "escuela", "universidad", "educación superior", "acceso a la universidad"],
    "Seguridad y Justicia": ["policía", "seguridad", "auxiliares", "dotación", "homicidio",
                            "delincuencia", "crimen", "atentado", "fallecimiento"],
    "Relaciones Internacionales": ["embajador", "cumbre", "celac", "diplomático", "internacional", "bilateral"],
    "Corrupción y Escándalos": ["escándalo", "corrupción", "investigan al", "denuncia contra", "fraude"],
    "Salud": ["salud", "hospital", "médico", "atención médica", "vacunación", "pandemia"],
    "Medio Ambiente": ["lluvias", "inundaciones", "emergencia", "clima", "ambiental", "desastre natural",
                      "huracán", "afectados", "coletazo"],
    "Economía": ["economía", "empleo", "empresa", "comercio", "inversión económica", "presupuesto"],
}

# ==================== FUNCIONES DE CLUSTERING (E5) ====================

def normalize_text(text):
    """Normaliza texto removiendo artículos y preposiciones"""
    t = text.lower()
    stopwords = ['el', 'la', 'los', 'las', 'un', 'una', 'unos', 'unas',
                 'de', 'del', 'en', 'para', 'por', 'con', 'sin', 'sobre',
                 'que', 'todo', 'está', 'son', 'es']

    t = re.sub(r'[^\wáéíóúñ\s]', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()

    words = t.split()
    words = [w for w in words if w not in stopwords and len(w) > 2]

    return " ".join(words)

def get_signature(text, words=20):
    """Genera firma textual normalizada para clustering"""
    normalized = normalize_text(text)
    return " ".join(normalized.split()[:words])

def universal_cluster(texts):
    """Clustering usando E5 (mantiene agrupación rápida y efectiva)"""
    print("📊 Agrupando noticias similares con E5...")
    groups = defaultdict(list)

    for i, t in enumerate(texts):
        signature = get_signature(t)
        groups[signature].append(i)

    representatives = []
    cluster_map = [-1] * len(texts)
    cid = 0

    for idxs in groups.values():
        longest_idx = max(idxs, key=lambda i: len(texts[i]))
        representatives.append(texts[longest_idx])

        for i in idxs:
            cluster_map[i] = cid
        cid += 1

    print(f"✅ {len(texts)} noticias → {len(representatives)} grupos únicos")
    return representatives, np.array(cluster_map)

# ==================== ANÁLISIS CON ROBERTA ====================

def get_roberta_sentiment(text, max_length=512):
    """Obtiene sentimiento base usando RoBERTa"""
    # Truncar texto si es muy largo
    text_truncated = text[:max_length * 4]  # Aproximado para tokens

    inputs = tokenizer(text_truncated, return_tensors="pt",
                      truncation=True, max_length=max_length,
                      padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = sentiment_model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)

    # BETO sentiment: 0=NEG, 1=NEU, 2=POS
    sentiment_idx = torch.argmax(probs, dim=1).item()
    confidence = probs[0][sentiment_idx].item()

    sentiment_map = {0: "Negativo", 1: "Neutro", 2: "Positivo"}

    return sentiment_map[sentiment_idx], confidence

def analyze_with_roberta(rep_text, client_name):
    """Análisis híbrido: RoBERTa + reglas contextuales"""
    lower = rep_text.lower()
    client_lower = client_name.lower()

    # PASO 1: Obtener sentimiento base de RoBERTa
    base_sentiment, base_confidence = get_roberta_sentiment(rep_text)

    # PASO 2: Ajustes contextuales específicos para marca

    # Caso 1: Negativos DIRECTOS contra el cliente (override RoBERTa)
    if any(pattern in lower for pattern in CLIENT_NEGATIVE_PATTERNS):
        return "Negativo", 0.95, base_sentiment

    # Caso 2: Reacciones neutras (override solo si RoBERTa marcó positivo/negativo)
    if any(reaction in lower for reaction in NEUTRAL_REACTIONS):
        # El cliente solo reacciona, no gestiona
        if base_sentiment != "Neutro":
            return "Neutro", 0.88, base_sentiment

    # Caso 3: POSITIVOS claros de gestión (reforzar si RoBERTa es neutro)
    client_is_positive_subject = any(pattern in lower for pattern in CLIENT_POSITIVE_PATTERNS)
    has_achievements = any(indicator in lower for indicator in ACHIEVEMENT_INDICATORS)

    if client_is_positive_subject or has_achievements:
        if base_sentiment == "Neutro":
            # RoBERTa no captó el positivo, pero el cliente SÍ está gestionando
            return "Positivo", 0.87, base_sentiment
        elif base_sentiment == "Positivo":
            # RoBERTa captó correctamente, aumentamos confianza
            return "Positivo", min(0.95, base_confidence + 0.10), base_sentiment

    # PASO 3: Usar análisis de RoBERTa como base
    return base_sentiment, round(base_confidence, 3), base_sentiment

def get_topic(text):
    """Análisis de tema por keywords"""
    lower = text.lower()
    tema = "Gestión y Acciones"
    max_matches = 0

    for topic, keywords in TOPICS_KEYWORDS.items():
        matches = sum(1 for kw in keywords if kw in lower)
        if matches > max_matches:
            max_matches = matches
            tema = topic

    return tema

# ==================== MAIN EXECUTION ====================
print("\n" + "="*80)
print("🤖 ANALIZADOR HÍBRIDO v31 - RoBERTa + E5")
print("="*80 + "\n")

client_name = input("👤 Nombre del cliente (ej: Carlos Pinedo): ").strip()
if not client_name:
    client_name = "Carlos Pinedo"
    print(f"   Usando cliente por defecto: {client_name}")

print("\n📂 Sube tu archivo Excel...")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

print(f"📖 Leyendo {file_name}...")
df = pd.read_excel(file_name)

text_columns = ['resumen', 'texto', 'contenido', 'titulo', 'title', 'noticia', 'descripcion']
text_col = next((c for c in text_columns if c in df.columns), None)

if not text_col:
    print(f"❌ Error: No se encontró columna de texto. Columnas disponibles: {list(df.columns)}")
    raise ValueError("No se encontró columna de texto válida")

print(f"✅ Columna detectada: '{text_col}'")

df = df.dropna(subset=[text_col]).reset_index(drop=True)
texts = df[text_col].astype(str).tolist()
print(f"📊 {len(texts)} noticias cargadas\n")

# ==================== PROCESAMIENTO ====================
t0 = time.time()

# PASO 1: Clustering con E5
rep_texts, cluster_map = universal_cluster(texts)

# PASO 2: Análisis con RoBERTa + reglas
print("\n🤖 Analizando tono con RoBERTa-spanish...")
tonos = []
confianzas = []
tonos_roberta_base = []  # Para comparación
temas = []

for rep in tqdm(rep_texts, desc="Procesando"):
    tono, conf, base_sent = analyze_with_roberta(rep, client_name)
    tema = get_topic(rep)

    tonos.append(tono)
    confianzas.append(conf)
    tonos_roberta_base.append(base_sent)
    temas.append(tema)

# PASO 3: Mapear a todas las noticias del grupo
print("\n🔗 Aplicando resultados a grupos similares...")
df["grupo_id"] = cluster_map
df["tono_marca"] = [tonos[cluster_map[i]] for i in range(len(df))]
df["confianza"] = [confianzas[cluster_map[i]] for i in range(len(df))]
df["tema"] = [temas[cluster_map[i]] for i in range(len(df))]
df["tono_roberta_base"] = [tonos_roberta_base[cluster_map[i]] for i in range(len(df))]

# ==================== EXPORTACIÓN ====================
out = f"Final_RoBERTa_{client_name.replace(' ', '_')}_{time.strftime('%Y%m%d_%H%M')}.xlsx"
print(f"\n💾 Guardando resultados en {out}...")

df.to_excel(out, sheet_name="Resultados", index=False, engine="openpyxl")
files.download(out)

# ==================== RESULTADOS ====================
tiempo_total = time.time() - t0
print(f"\n{'='*80}")
print(f"✅ ¡ANÁLISIS COMPLETADO CON RoBERTa!")
print(f"⏱️  Tiempo: {tiempo_total:.1f}s")
print(f"📄 Archivo: {out}")
print(f"{'='*80}\n")

total = len(df)
pos = (df["tono_marca"] == "Positivo").sum()
neg = (df["tono_marca"] == "Negativo").sum()
neu = (df["tono_marca"] == "Neutro").sum()

# Comparación: ajustes aplicados
ajustes = (df["tono_marca"] != df["tono_roberta_base"]).sum()

print("📊 RESUMEN:\n")
print(f"   📈 Total noticias: {total}")
print(f"   🔢 Grupos únicos: {len(rep_texts)}")
print(f"   🔗 Promedio noticias/grupo: {total/len(rep_texts):.1f}")
print(f"   ✅ Positivas: {pos} ({pos/total*100:.1f}%)")
print(f"   ❌ Negativas: {neg} ({neg/total*100:.1f}%)")
print(f"   ⚪ Neutras: {neu} ({neu/total*100:.1f}%)")
print(f"\n   🔧 Ajustes contextuales aplicados: {ajustes} ({ajustes/total*100:.1f}%)")

print("\n📈 DISTRIBUCIÓN DE TONO FINAL:")
print(df["tono_marca"].value_counts().to_string())

print("\n🤖 TONO BASE ROBERTA (sin ajustes):")
print(df["tono_roberta_base"].value_counts().to_string())

print("\n🎯 DISTRIBUCIÓN DE TEMAS:")
print(df["tema"].value_counts().to_string())

# Mostrar ejemplos de ajustes
if ajustes > 0:
    print("\n🔍 EJEMPLOS DE AJUSTES CONTEXTUALES:")
    adjusted = df[df["tono_marca"] != df["tono_roberta_base"]].head(3)
    for idx, row in adjusted.iterrows():
        texto_corto = row[text_col][:90] + "..." if len(row[text_col]) > 90 else row[text_col]
        print(f"\n   📰 {texto_corto}")
        print(f"      RoBERTa base: {row['tono_roberta_base']} → Final: {row['tono_marca']}")

print("\n✨ Análisis finalizado exitosamente ✨")

🚀 Dispositivo: CUDA
📦 Cargando multilingual-e5-large para clustering...
📦 Cargando RoBERTa-spanish para análisis de tono...


tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Procesando:   0%|          | 0/1210 [00:00<?, ?it/s]


🔗 Aplicando resultados a grupos similares...

💾 Guardando resultados en Final_RoBERTa_Carlos_Pinedo_20251118_0236.xlsx...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ ¡ANÁLISIS COMPLETADO CON RoBERTa!
⏱️  Tiempo: 15.6s
📄 Archivo: Final_RoBERTa_Carlos_Pinedo_20251118_0236.xlsx

📊 RESUMEN:

   📈 Total noticias: 1432
   🔢 Grupos únicos: 1210
   🔗 Promedio noticias/grupo: 1.2
   ✅ Positivas: 460 (32.1%)
   ❌ Negativas: 125 (8.7%)
   ⚪ Neutras: 847 (59.1%)

   🔧 Ajustes contextuales aplicados: 119 (8.3%)

📈 DISTRIBUCIÓN DE TONO FINAL:
tono_marca
Neutro      847
Positivo    460
Negativo    125

🤖 TONO BASE ROBERTA (sin ajustes):
tono_roberta_base
Neutro      946
Positivo    353
Negativo    133

🎯 DISTRIBUCIÓN DE TEMAS:
tema
Medio Ambiente                510
Gestión y Acciones            349
Infraestructura               193
Educación                     134
Economía                      111
Seguridad y Justicia           74
Relaciones Internacionales     40
Salud                          19
Corrupción y Escándalos         2

🔍 EJEMPLOS DE AJUSTES CONTEXTUALES:

   📰 Alcaldía de Santa Marta presenta proyecto de atención integral para más de 2 mil adulto